# 20220324: Comparing `fastparquet` to `sqlite`
Timings and data storage sizes for Pandas dataframes generated from NetCDF files

In [1]:
import iris
import pandas as pd
import fastparquet as fpq
import sqlite3 as sql
import irisxarraypandas as ixp

In [2]:
%%time
fname = "pr_ukcp_1yr"
cube = iris.load_cube(f"{fname}.nc")
cube.summary(shorten=True)

CPU times: user 77.7 ms, sys: 14.6 ms, total: 92.3 ms
Wall time: 105 ms


'lwe_precipitation_rate / (mm/day)   (ensemble_member: 1; time: 360; projection_y_coordinate: 112; projection_x_coordinate: 82)'

In [3]:
%%time
df = ixp.cube_to_dataframe(ixp.add_lat_lon(cube))

/Users/kevin.donkers/opt/miniconda3/envs/data/lib/python3.7/site-packages/iris/coord_systems.py:531: UserWarning: The default value for the *approx* keyword argument to TransverseMercator will change from True to False after 0.18.
  globe=globe,


CPU times: user 7.05 s, sys: 1.49 s, total: 8.55 s
Wall time: 8.74 s


In [4]:
# SQLite and Parquet were struggling to process the time objects
df.drop(columns=['time'], inplace=True)

In [5]:
df.head()

,ensemble_member,projection_y_coordinate,projection_x_coordinate,ensemble_member_id,month_number,year,yyyymmdd,grid_latitude,grid_longitude,latitude,longitude,pr
0,1,-102000.0,-210000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.432699,-18.241456,48.682767,-10.291939,0.690706
1,1,-102000.0,-198000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.404462,-18.138041,48.694350,-10.130549,0.853663
2,1,-102000.0,-186000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.376215,-18.034611,48.705708,-9.969056,1.088705
3,1,-102000.0,-174000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.347959,-17.931165,48.716841,-9.807464,1.452076
4,1,-102000.0,-162000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.319692,-17.827705,48.727750,-9.645773,1.969763


# CSV

Write

In [6]:
%%time
df.to_csv(f"{fname}.csv")

CPU times: user 26.5 s, sys: 728 ms, total: 27.3 s
Wall time: 28.3 s


Read

In [13]:
%%time
pd.read_csv(f"{fname}.csv").head()

CPU times: user 4.12 s, sys: 822 ms, total: 4.94 s
Wall time: 5.11 s


,Unnamed: 0,ensemble_member,projection_y_coordinate,projection_x_coordinate,ensemble_member_id,month_number,year,yyyymmdd,grid_latitude,grid_longitude,latitude,longitude,pr
0,0,1,-102000.0,-210000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.432699,-18.241456,48.682767,-10.291939,0.690706
1,1,1,-102000.0,-198000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.404462,-18.138041,48.694350,-10.130549,0.853663
2,2,1,-102000.0,-186000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.376215,-18.034611,48.705708,-9.969056,1.088705
3,3,1,-102000.0,-174000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.347959,-17.931165,48.716841,-9.807464,1.452076
4,4,1,-102000.0,-162000.0,HadREM3-GA705-r001i1p00000,12,2020,20201201,1.319692,-17.827705,48.727750,-9.645773,1.969763


## SQLite

Write

In [8]:
%%time
conn = sql.connect(f"{fname}.db")
df.to_sql('ukcp18', conn)

CPU times: user 10 s, sys: 1.36 s, total: 11.4 s
Wall time: 12.1 s


Read

In [9]:
%%time
pd.read_sql('SELECT yyyymmdd, pr FROM ukcp18', conn).head()

CPU times: user 3.25 s, sys: 860 ms, total: 4.11 s
Wall time: 5.91 s


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


## Parquet

Write

In [10]:
%%time
fpq.write(f"{fname}.parq", df)

CPU times: user 2.35 s, sys: 461 ms, total: 2.81 s
Wall time: 3.07 s


Read

In [11]:
%%time
fpq.ParquetFile(f"{fname}.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 183 ms, sys: 23.8 ms, total: 207 ms
Wall time: 213 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With GZIP compression

Write

In [14]:
%%time
fpq.write(f"{fname}_gzip.parq", df, compression="GZIP")

CPU times: user 7.72 s, sys: 516 ms, total: 8.24 s
Wall time: 8.46 s


Read

In [15]:
%%time
fpq.ParquetFile(f"{fname}_gzip.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 308 ms, sys: 69.6 ms, total: 378 ms
Wall time: 380 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With SNAPPY compression

Write

In [17]:
%%time
fpq.write(f"{fname}_snappy.parq", df, compression="SNAPPY")

CPU times: user 2.47 s, sys: 605 ms, total: 3.07 s
Wall time: 3.13 s


Read

In [18]:
%%time
fpq.ParquetFile(f"{fname}_snappy.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 220 ms, sys: 80 ms, total: 300 ms
Wall time: 301 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With brotli compression

Write

In [19]:
%%time
fpq.write(f"{fname}_brotli.parq", df, compression="brotli")

CPU times: user 1min 11s, sys: 1.41 s, total: 1min 12s
Wall time: 1min 13s


Read

In [20]:
%%time
fpq.ParquetFile(f"{fname}_brotli.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 431 ms, sys: 110 ms, total: 542 ms
Wall time: 548 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With lz4 compression

Write

In [21]:
%%time
fpq.write(f"{fname}_lz4.parq", df, compression="lz4")

CPU times: user 2.39 s, sys: 502 ms, total: 2.89 s
Wall time: 2.99 s


Read

In [22]:
%%time
fpq.ParquetFile(f"{fname}_lz4.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 232 ms, sys: 72 ms, total: 304 ms
Wall time: 304 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With lz4_raw compression

Write

In [30]:
%%time
fpq.write(f"{fname}_lz4_raw.parq", df, compression="lz4_raw")

CPU times: user 2.38 s, sys: 484 ms, total: 2.86 s
Wall time: 3.14 s


Read

In [31]:
%%time
fpq.ParquetFile(f"{fname}_lz4_raw.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 230 ms, sys: 69.9 ms, total: 300 ms
Wall time: 300 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


### With zstandard compression

Write

In [26]:
%%time
fpq.write(f"{fname}_zstd.parq", df, compression="zstd")

CPU times: user 2.64 s, sys: 622 ms, total: 3.26 s
Wall time: 3.31 s


Read

In [27]:
%%time
fpq.ParquetFile(f"{fname}_zstd.parq").to_pandas(['yyyymmdd', 'pr']).head()

CPU times: user 264 ms, sys: 89.1 ms, total: 353 ms
Wall time: 357 ms


,yyyymmdd,pr
0,20201201,0.690706
1,20201201,0.853663
2,20201201,1.088705
3,20201201,1.452076
4,20201201,1.969763


## How big are the resulting files?

In [34]:
!ls -lh {fname}*

-rw-r--r--  1 kevin.donkers  staff   494M 24 Mar 17:29 pr_ukcp_1yr.csv
-rw-r--r--  1 kevin.donkers  staff   384M 24 Mar 17:30 pr_ukcp_1yr.db
-rw-r--r--  1 kevin.donkers  staff    13M 24 Mar 16:40 pr_ukcp_1yr.nc
-rw-r--r--  1 kevin.donkers  staff   347M 24 Mar 17:30 pr_ukcp_1yr.parq
-rw-r--r--  1 kevin.donkers  staff    12M 24 Mar 17:43 pr_ukcp_1yr_brotli.parq
-rw-r--r--  1 kevin.donkers  staff   105M 24 Mar 17:37 pr_ukcp_1yr_gz.parq
-rw-r--r--  1 kevin.donkers  staff   115M 24 Mar 17:43 pr_ukcp_1yr_lz4.parq
-rw-r--r--  1 kevin.donkers  staff   115M 24 Mar 17:45 pr_ukcp_1yr_lz4_raw.parq
-rw-r--r--  1 kevin.donkers  staff   125M 24 Mar 17:42 pr_ukcp_1yr_snappy.parq
-rw-r--r--  1 kevin.donkers  staff    12M 24 Mar 17:44 pr_ukcp_1yr_zstd.parq
